In [54]:
import numpy as np
import matplotlib.pyplot as plt

In [94]:
def calc_mutual_inductance(coil, d=0, po=0, fi=0):
    """Расчёт взаимной индуктивности между двумя витками.
    :param coil_1: np.ndarray с радиусом одной катушки индуктивности, м
    :type coil_1: np.ndarray
    
    :param coil_2: список с радиусом одной катушки индуктивности, м
    :type coil_1: np.ndarray
    
    :param d: расстояние между витками, м
    :type d: np.ndarray or float, int
    
    :param po: латеральное смещение между витками, м
    :type po: np.ndarray or float, int
    
    :param fi: угловое смещение между витками
    :type fi: np.ndarray or float, int
    
    :rtype: np.ndarray
    :return: M, собственная индуктивность между витками
    """
    mu0 = 4 * np.pi * 10 ** (-7)
    M = np.array([])
    m = 0
    N = 60
    K = 60
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ind_ri in range(len(coil)):
        for ind_rj in range(ind_ri + 1, len(coil)):
            m = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil[ind_ri] * np.cos(df2 * k) * np.cos(fi) - coil[ind_rj] * np.cos(df1 * n)
                    yk_yn = coil[ind_ri] * np.sin(df2 * k) * np.cos(fi) - coil[ind_rj] * np.sin(df1 * n)
                    zk_zn = d + coil[ind_rj] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    m += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            m *= mu0 * coil[ind_ri] * coil[ind_rj] / (4 * np.pi)
            M = np.append(M, m)
    return M


def calc_self_inductance(coil, thin=0.0004):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = 0
    for i in range(len(coil)):
        L += mu0 * coil[i] * (np.log(8 * coil[i] / thin) - 7 / 4 + (thin ** 2) / (8 * coil[i] ** 2) * (np.log(8 * coil[i] / thin) + 1 / 3))
    L += 2 * np.sum(calc_mutual_inductance(coil))
    return L

Reproduction the result from article: Self-inductance of Multi-turn Round Inductive Coils with Production-specific Geometric Features

In [95]:
case1 = np.linspace(0.0225, 0.0475, 6, endpoint=True)
case2 = np.linspace(0.024, 0.048, 7, endpoint=True)
case3 = np.linspace(0.024, 0.048, 4, endpoint=True)
case4 = np.linspace(0.02, 0.044, 3, endpoint=True)
case5 = np.linspace(0.02, 0.06, 11, endpoint=True)
case6 = np.linspace(0.0035, 0.02, 12, endpoint=True)
case7 = np.linspace(0.005, 0.02, 7, endpoint=True)
print("case1:", calc_self_inductance(case1)*10**6, "мкГн")
print("case2:", calc_self_inductance(case2)*10**6, "мкГн")
print("case3:", calc_self_inductance(case3)*10**6, "мкГн")
print("case4:", calc_self_inductance(case4)*10**6, "мкГн")
print("case5:", calc_self_inductance(case5)*10**6, "мкГн")
print("case6:", calc_self_inductance(case6, thin=0.00025)*10**6, "мкГн")
print("case7:", calc_self_inductance(case7, thin=0.00025)*10**6, "мкГн")

case1: 3.089794194386572 мкГн
case2: 4.397392173817378 мкГн
case3: 1.5415228555041434 мкГн
case4: 0.7925535409403592 мкГн
case5: 9.897668603813843 мкГн
case6: 2.7808388538242848 мкГн
case7: 1.1303573692401938 мкГн
